## Libraries

In [5]:
import os
import cv2
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
from shutil import copyfile
from glob import glob
from multiprocessing import Pool

## Label Colors

In [6]:
labels={"playera_gris/playera_blanca":(255,0,0),        
         "playera_negra/playera_azul":(0,255,0),
         "pantalon_mezclilla":(0,0,255),
         "pantalon_negro":(0,255,255),
         "sudadera_azul":(255,255,0),
         "playera-tigre":(255,0,255)}

## Generate Train DataSet

In [7]:
img_paths=glob("../../data/LookBook-Labeled/labeled_imgs/*.jpg")
gen=iter(range(1000))
for img_path in img_paths:
    img=plt.imread(img_path).copy()
    mask_paths=glob(img_path.replace(".jpg","")+"_*.png")
    for mask_path in mask_paths:
        img=plt.imread(img_path).copy()
        mask=(plt.imread(mask_path)[...,:3]*255).astype(np.uint8)
        h,w,_=np.where(mask!=0)
        img[(mask[...,0]==0)&(mask[...,1]==0)&(mask[...,2]==0)]=[255,255,255]
        img=img[min(h):max(h),min(w):max(w)]
        mask=mask[min(h):max(h),min(w):max(w)]
        merge_img=cv2.resize(np.concatenate([img,mask],axis=1),(256*2,256))

        plt.imsave(f"../../data/LookBook-Labeled/train/{next(gen)}.png",merge_img)
        degree=np.random.randint(0,180)
        merge_img=cv2.resize(np.concatenate([np.array(Image.fromarray(img).rotate(degree,fillcolor=(255,255,255))),
                                             np.array(Image.fromarray(mask).rotate(degree,fillcolor=(0,0,0)))],axis=1),(256*2,256))
        
        plt.imsave(f"../../data/LookBook-Labeled/train/{next(gen)}.png",merge_img)
        
        merge_img=cv2.resize(np.concatenate([np.fliplr(img),np.fliplr(mask)],axis=1),(256*2,256))
        
        merge_img=cv2.resize(np.concatenate([np.fliplr(img),np.fliplr(mask)],axis=1),(256*2,256))
        plt.imsave(f"../../data/LookBook-Labeled/train/{next(gen)}.png",merge_img)
        
        merge_img=cv2.resize(np.concatenate([np.flipud(img),np.flipud(mask)],axis=1),(256*2,256))
        plt.imsave(f"../../data/LookBook-Labeled/train/{next(gen)}.png",merge_img)

## Train Val Split

In [9]:
train_files=glob("../../data/LookBook-Labeled/train/*")

np.random.shuffle(train_files)

for src in train_files[:int(len(train_files)*.3)]:
    os.rename(src,src.replace("train","val"))
    